In [3]:
%%writefile assignment32.cu
// Задание 2
// Релизовать CUDA-программу для плэлементного сложения двух массовов
// Исследовать влиние размера блока потоков на производительность
// Провести замеры времени дл минимум трех размеров блока
#include <iostream>                      // Для работы с вводом/выводом
#include <cuda_runtime.h>                // Основные функции CUDA
#include <chrono>                        // Для измерения времени выполнения

using namespace std;                     // Чтобы не писать std:: перед cout

// Поэлементное сложение
__global__ void vector_add(float *A, float *B, float *C, int n)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Вычисляем глобальный индекс потока

    if (idx < n)                                     // Проверяем, что не вышли за границы массива
    {
        C[idx] = A[idx] + B[idx];                    // Складываем элементы двух массивов
    }
}

int main()                                           // Основная функция
{
    int n = 1000000;                                 // Размер массивов
    size_t size = n * sizeof(float);                 // Размер массива в байтах

    // Выделение памяти на CPU
    float *h_A = new float[n];                       // Первый массив на CPU
    float *h_B = new float[n];                       // Второй массив на CPU
    float *h_C = new float[n];                       // Результирующий массив на CPU

    for (int i = 0; i < n; i++)                      // Заполняем входные массивы
    {
        h_A[i] = 1.0f;                               // Присваиваем всем A[i] = 1.0
        h_B[i] = 2.0f;                               // Присваиваем всем B[i] = 2.0
    }

    // Выделение памяти на GPU
    float *d_A, *d_B, *d_C;                          // Указатели на память GPU

    cudaMalloc(&d_A, size);                          // Выделяем память под массив A на GPU
    cudaMalloc(&d_B, size);                          // Выделяем память под массив B на GPU
    cudaMalloc(&d_C, size);                          // Выделяем память под массив C на GPU

    // Копирование данных CPU -> GPU
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice); // Копируем A в память GPU
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice); // Копируем B в память GPU

    // Тестируем разные размеры блока
    int blockSizes[3] = {128, 256, 512};             // Три варианта размера блока

    for (int i = 0; i < 3; i++)                      // Цикл по разным конфигурациям
    {
        int blockSize = blockSizes[i];               // Текущий размер блока
        int gridSize = (n + blockSize - 1) / blockSize; // Вычисляем количество блоков

        cudaDeviceSynchronize();                     // Дожидаемся завершения предыдущих операций
        auto start = chrono::high_resolution_clock::now(); // Начало замера

        vector_add<<<gridSize, blockSize>>>(d_A, d_B, d_C, n); // Запуск CUDA kernel

        cudaDeviceSynchronize();                     // Ожидаем завершения kernel
        auto end = chrono::high_resolution_clock::now(); // Конец замера

        chrono::duration<double> elapsed = end - start; // Вычисление продолжительность

        cout << "Размер блока: " << blockSize            // Выводим размер блока
             << " | Время: " << elapsed.count()         // Выводим время выполнения
             << " с" << endl;
    }


    // Копирование результата обратно
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost); // Копируем результат GPU -> CPU

    // Освобождение памяти
    cudaFree(d_A);                                  // Освобождаем память GPU
    cudaFree(d_B);
    cudaFree(d_C);

    delete[] h_A;                                   // Освобождаем память CPU
    delete[] h_B;
    delete[] h_C;

    return 0;                                       // Завершение
}


Overwriting assignment32.cu


In [4]:
!nvcc assignment32.cu -o assignment32
!./assignment32


Размер блока: 128 | Время: 0.0075569 с
Размер блока: 256 | Время: 2.16e-06 с
Размер блока: 512 | Время: 1.461e-06 с
